In [36]:

with open("SimpleStorage.sol","r") as file:
    simple_storage_file = file.read()
    #print(simple_storage_file)

# Steps
pip install web3
pip install black
pip install py-solc-x
solcx.install_solc('0.6.0')

In [41]:
from solcx import compile_standard
import json
compiled_sol = compile_standard(
    {
    "language": "Solidity",
    "sources": {"SimpleStorage.sol" : {"content": simple_storage_file}},
    "settings": {
        "outputSelection" : {
            "*" : {
                "*" : ["abi","metadata","evm.bytecode","evm.sourceMap"]}
        }
    },
    },
    solc_version = "0.6.0"
    )

with open("compilied_code.json", "w") as file:
    json.dump(compiled_sol, file)

#get bytecode
bytecode = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["evm"]["bytecode"]["object"]
#get abi
abi = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["abi"]

 